# Maximum Causal Entropy Inverse Reinforcement Learning (MCE IRL)

Maximum Causal Entropy (MCE) IRL [[Ziebart et al., 2010](https://www.cs.cmu.edu/~bziebart/publications/thesis-bziebart.pdf)] is one of popular IRL algorithms, and has much affected a lot of today's works (such as, [GAIL](./06_gail.ipynb)) in IRL.

The previous [Maximum Entropy IRL](./03_maxent_irl.ipynb) algorithm is based on the conditional entropy $-\sum_s \sum_a P(a, s) log P(a|s)$, which assumes the action $a$ is conditioned only on state $s$.<br>
On contrary, Maximum Causal Entropy (MCE) IRL applies the causality of time-steps by introducing the causally conditioned probability $\prod_{t=0}^{T-1} P(a_t | s_{0:t}, a_{0:t-1})$.<br>
In MCE IRL, the following **causal entropy** with a discount $\gamma$ is applied instead.

$\displaystyle H(A_{0:T-1} || S_{0:T-1}) = -\sum_{t=0}^{T-1} \sum_{a_{0:t},x_{0:t}} \gamma^t P(a_{0:t},s_{0:t}) \log(\pi(a_t | s_{0:t}, a_{0:t-1})) = \mathbb{E}_{\pi} \left[ -\sum_{t=0}^{T-1} \gamma^t \log \pi_t(a_t | s_t) \right]$

Especially when the transition probability is stochastic (i.e, stochastic dynamics), this time-step causality (i.e, causally conditioned probability) makes sense.

> Note : When it's deterministic dynamics (i.e, transition probability has ```1.0``` for one successor state and ```0.0``` for all others), it's known that maximizing the causal entropy reduces to a conditional entropy maximization. (MCE IRL is then simplified to Maximum Entropy IRL.)<br>
> See Theorem 4.22 in [original paper](https://www.cs.cmu.edu/~bziebart/publications/thesis-bziebart.pdf).

Because Maximum Entropy IRL algorithm (see [here](./03_maxent_irl.ipynb)) has a loss of time-steps causality in the conditional entropy, it suffers by needing a bias term for the transition probabilities.<br>
Maximum Causal Entropy (MCE) IRL fixes this problem by applying the causal entropy instead.

> Note : In this example, however, the transition probability is deterministic. Here I apply MCE RL also in this example for your learning purpose.

*(back to [index](https://github.com/tsmatz/imitation-learning-tutorials/))*

In [ ]:
import numpy as np
print(np.__version__)

2.0.2


In [ ]:
from IPython.display import clear_output
clear_output()

In [ ]:
get_ipython().system('pip install torch numpy')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [ ]:
# Clone from Github Repository
! git init .
! git remote add origin https://github.com/RichardMinsooGo-RL-Gym/Imitation-learning-Gridworld.git
! git pull origin master
# ! git pull origin main

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 141, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 141 (delta 90), reused 76 (delta 34), pack-reused 0 (from 0)
Receiving objects: 100% (141/141), 1.45 MiB | 13.04 MiB/s, done.
Resolving deltas: 100% (90/90), done.
From https://github.com/tsmatz/imitation-learning-tutorials
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> origin

## Overview of Maximum Causal Entropy Inverse Reinforcement Learning method

Here I'll follow the paper "[A Primer on Maximum Causal Entropy Inverse Reinforcement Learning](https://arxiv.org/pdf/2203.11409)", which is the compressed (concise) derivation of MCE IRL algorithm.

In this example, here I assume that the number of time-step (horizon) $T$ is finite.

As I have discussed in [previous example](./03_maxent_irl.ipynb), we consider to find policy $\pi$ :

$\displaystyle \max_{\pi} H(A_{0:T-1} || S_{0:T-1}) $

subject to the following constraint (i.e, feature expectation matching) :

$\displaystyle \mathbb{E}_{\pi} \left[ \sum_{t=0}^{T-1} \gamma^t \phi(S_t,A_t) \right] = \mathbb{E}_{\pi^{\ast}} \left[ \sum_{t=0}^{T-1} \gamma^t \phi(S_t,A_t) \right]$

where $\pi^{\ast}$ is expert policy.

> Note : See above definition for $H(A_{0:T-1} || S_{0:T-1})$.

Same as in [previous example](./03_maxent_irl.ipynb), we solve this primal problem by applying Lagrangian, making dual problem, solving dual problem with KKT conditions, and optimizing parameters with gradient methods.

Lagrangian $\Lambda(\pi, \theta)$ is formed as follows.

$\displaystyle \Lambda(\pi, \theta)=H(A_{0:T-1}||S_{0:T-1}) + \theta^T \cdot \left( \mathbb{E}_{\pi} \left[ \sum_{t=0}^{T-1} \gamma^t \phi(S_t,A_t) \right] - \mathbb{E}_{\pi^{\ast}} \left[ \sum_{t=0}^{T-1} \gamma^t \phi(S_t,A_t) \right] \right) \;\;\;\;\;\;\ (1)$

By Lagrangian duality, this problem is to find :

$\displaystyle \min_{\theta} \left( \max_{\pi} \Lambda(\pi, \theta) \right) \;\;\;\;\;\;\ (2)$

MCE IRL method is to alternately update $\pi$ and $\theta$ as follows. :

- Update $\pi$ to maximize $\Lambda(\pi, \theta)$.
- Update $\theta$ to minimize above (2)

Now we focus on the nested $\max_{\pi} \Lambda(\pi, \theta)$ for a given $\theta$.

Recall that $\pi(a_t|s_t)$ has the following constraints. :

$\pi(a_t|s_t) \geq 0$ and $\sum_{a_t} \pi(a_t|s_t) = 1$

Therefore, when we denote $h_{s_t}(\pi) = \sum_{a_t} \pi(a_t|s_t) - 1$, this problem can be expressed as follows. :

$\displaystyle \max_{\pi} \Lambda(\pi, \theta)$

subject to :

$h_{s_t}(\pi) = 0 \;\;\;\;\;\; \forall t = 0, 1, \ldots , T-1 \;\;\; \forall s_t$

By applying Lagrangian again, this problem can be written as:

$\displaystyle \psi(\pi, \mu, \theta) = \Lambda(\pi, \theta) + \sum_{s_t, t} \mu_{s_t} h_{s_t}(\pi)$

with the following conditions. :

$\displaystyle \nabla_{\pi} \psi(\pi, \mu, \theta)=0 $

$\displaystyle h_{s_t}(\pi) = 0 \;\;\;\; (\forall t = 0, 1, \ldots , T-1 \;\;\; \forall s_t) $

It's known that both conditions (KKT conditions) are satisfied by the following policy $\pi(a_t,s_t)$ :

$\displaystyle \pi_t(a_t | s_t) = \exp \left( Q_{\theta, t}^{\verb|soft|}(s_t, a_t) - V_{\theta, t}^{\verb|soft|}(s_t) \right)$

in which, $Q_{\theta, t}^{\verb|soft|}(S_t, A_t)$ and $V_{\theta, t}^{\verb|soft|}(S_t)$ are recursively obtained as follows. :

- $\displaystyle V_{\theta, t}^{\verb|soft|}(s_t) = \log \sum_{a_t} \exp  Q_{\theta, t}^{\verb|soft|}(s_t, a_t) \;\;\; (\forall t = 0,1,\ldots,T-1) $
- $\displaystyle Q_{\theta, t}^{\verb|soft|}(s_t, a_t) = \theta^T \cdot \phi(s_t, a_t) + \gamma \mathbb{E}_{\mathcal{T}} \left[ V_{\theta,t+1}^{\verb|soft|}(S_{t+1}) | s_t, a_t \right] \;\;\; (\forall t = 0,1,\ldots,T-2) $
- $\displaystyle  Q_{\theta, T-1}^{\verb|soft|}(s_{T-1}, a_{T-1}) = \theta^T \cdot \phi(s_{T-1},a_{T-1}) $

> Note : See the paper "[A Primer on Maximum Causal Entropy Inverse Reinforcement Learning](https://arxiv.org/pdf/2203.11409)" for this proof.<br>
> This satisfies : $\sum_{a_t} \pi_t(a_t | s_t) = 1.0 \;\;\; \forall t, s_t$

Here $\mathbb{E}_{\mathcal{T}}$ is the expectation to transit states with known transition probability.<br>
Thus, such like previous Maximum Entropy IRL, the state transition distribution should also be known in this method. (See the [next example](./05_relent_irl.ipynb) about approximation for the unknown transition.)

In abstraction, $Q^{\verb|soft|}(s_t,a_t)$ and $V^{\verb|soft|}(s_t)$ corresponds to the traditional Q-value and Value in regular reinforcement learning respectively (see [here](https://github.com/tsmatz/reinforcement-learning-tutorials/blob/master/03-actor-critic.ipynb)), but it's log-sum-exp values, not values itself. (This is the reason why the name implies "soft".)

Next we consider to update $\theta$ by applying gradient descent with respect to $\theta$.<br>
Form equation (1), the gradient (nabla) is obtained by :

$\displaystyle \nabla_{\theta} \Lambda(\pi, \theta) = \mathbb{E}_{\pi} \left[ \sum_{t=0}^{T-1} \gamma^t \phi(S_t,A_t) \right] - \mathbb{E}_{\pi^{\ast}} \left[ \sum_{t=0}^{T-1} \gamma^t \phi(S_t,A_t) \right]$

The second term is obtained by expert demonstrations.<br>
The first term is computed by the probability of initial states, the policy $\pi$ (which is obtained above), and transition probability.

As you will see in [GAIL example](./06_gail.ipynb) (generative adversarial imitation learning) later, this method is refined to non-linear rewards.

## Implementation

Now let's start implementation.

> Note : To speed up computation, I'll implement operations with PyTorch tensors.

### 1. Restore environment and load expert's data

Before we start, we need to install the required packages.

In [ ]:
!pip install torch numpy

Firstly, I restore GridWorld environment from JSON file.

For details about this environment, see [Readme.md](https://github.com/tsmatz/imitation-learning-tutorials/blob/master/Readme.md).

> Note : See [this script](./00_generate_expert_trajectories.ipynb) for generating the same environment.

In [ ]:
import torch
import json
from gridworld import GridWorld

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open("gridworld.json", "r") as f:
    json_object = json.load(f)
    env = GridWorld(**json_object, device=device)

Now I visualize our GridWorld environment.

The number in each cell indicates the reward score on this state.<br>
The goal state is on the right-bottom corner (in which the reward is ```10.0```), and the initial state is uniformly picked up from the gray-colored cells.<br>
If the agent can reach to goal state without losing any rewards, it will get ```10.0``` for total reward.

See [Readme.md](https://github.com/tsmatz/imitation-learning-tutorials/blob/master/Readme.md) for details about the game rule of this environment.

In [ ]:
from IPython.display import HTML, display

valid_states_all = torch.cat((env.valid_states, torch.tensor([env.grid_size-1,env.grid_size-1]).to(device).unsqueeze(dim=0)))
valid_states_all = valid_states_all[:,0] * env.grid_size + valid_states_all[:,1]

html_text = "<table>"
for row in range(env.grid_size):
    html_text += "<tr>"
    for col in range(env.grid_size):
        if row*env.grid_size + col in valid_states_all:
            html_text += "<td bgcolor=\"gray\">"
        else:
            html_text += "<td>"
        html_text += str(env.reward_map[row*env.grid_size+col].tolist())
        html_text += "</td>"
    html_text += "</tr>"
html_text += "</table>"

display(HTML(html_text))

0,-1,0,0,0,-1,0,0,0,0,0,0,-1,0,-1,-1,-1,-1,0,-1,0,-1,0,0,0,-1,0,0,0,-1,-1,0,-1,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0
-1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,0,-1,0,0,0,0,0,-1,0,-1,0,-1,0,-1,0,-1,-1,0,0,0,-1,-1,0,0,-1,-1
0,0,-1,-1,0,-1,0,-1,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,-1,-1,0,0,-1,0,0,0,0,-1,0,0,0
0,0,-1,0,-1,-1,0,0,-1,0,-1,0,0,0,0,-1,0,-1,0,-1,0,-1,0,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1
-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,-1,0,0,-1,0,-1,-1,0,0
0,-1,-1,0,0,-1,0,-1,0,0,-1,0,0,-1,0,0,-1,-1,0,0,-1,0,0,0,-1,0,0,0,0,-1,-1,0,0,-1,0,-1,-1,-1,-1,0,0,-1,0,-1,0,0,0,0,0,-1
-1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,-1,-1,0,0,-1,-1,0,0,-1,-1,0,-1,0,0,0,0,0,0,0,-1,-1,0,0,0,-1,0,-1,0,-1,-1
0,0,0,0,-1,0,0,0,-1,0,0,0,0,-1,0,0,-1,-1,0,0,-1,0,0,-1,0,0,-1,0,0,0,-1,-1,0,0,-1,0,-1,0,-1,0,0,-1,0,-1,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,-1,-1,0,-1,0,0,-1,0,0,-1,-1,0,0,0,0,0,0,-1,0,0,0,-1,-1,-1,-1,-1,-1,0,-1,-1,0,-1,0,0,0,-1,0,0,0,0
0,0,0,-1,0,-1,0,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,-1,0,-1,-1,-1
0,0,0,0,0,-1,0,0,0,0,-1,0,0,-1,0,0,0,-1,0,0,0,-1,0,0,-1,0,0,0,-1,-1,0,-1,-1,0,0,0,0,0,-1,-1,-1,-1,-1,-1,0,0,-1,-1,-1,0


Load expert's data (demonstrations) which is saved in ```./expert_data``` folder in this repository.

> Note : See [this script](./00_generate_expert_trajectories.ipynb) for generating expert dataset.

In [ ]:
import pickle

dest_dir = "./expert_data"
checkpoint_file = "ckpt0.pkl"

# load expert data from pickle
with open(f"{dest_dir}/{checkpoint_file}", "rb") as f:
    exp_data = pickle.load(f)
exp_states = exp_data["states"]
exp_actions = exp_data["actions"]
timestep_lens = exp_data["timestep_lens"]

### 2. Create a function to build feature vector $\phi(s,a)$

First I create a function to build feature vector $\phi(s,a)$.

In this example, we simply build an one-hot vector $\phi(s,a)$, which has flatten elements of a matrix ``````(STATE_SIZE, ACTION_SIZE)``````, in which $s$-th row and $a$-th column is one, and zeros for all others. (Therefore the number of one-hot's elements is ```STATE_SIZE * ACTION_SIZE```.)<br>
For instance, $\phi(3, 2)$ has ```1.0``` in 14th element and ```0.0``` for others, because ```3 x 4 + 2 = 14```. (Here ```4``` is the size of available actions.)

This feature is valid, because the reward becomes linear to this feature vector $\phi(s,a)$, i.e, $\verb|reward| = \theta^T \cdot \phi(s,a)$.

In [ ]:
from torch.nn import functional as F

STATE_SIZE = env.grid_size*env.grid_size  # 2500
ACTION_SIZE = env.action_size             # 4

def get_features(s, a):
    """
    Return feature vectors corresponding states s and actions a.

    Parameters
    ----------
    s : torch.tensor((...), dtype=int)
        Tensor of state id.
        This can have arbitrary shape, but the shape must be same as the shape of a.
    a : torch.tensor((...), dtype=int)
        Tensor of action id.
        This can have arbitrary shape, but the shape must be same as the shape of s.

    Returns
    ----------
    torch.tensor((..., STATE_SIZE*ACTION_SIZE), dtype=int)
        Generated feature vectors.
        When the shape of arguments is (*), the shape of returned result is (*, 2500*4).
    """
    assert s.size() == a.size()
    ids = s * ACTION_SIZE + a
    return F.one_hot(ids, num_classes=STATE_SIZE*ACTION_SIZE).float()

Now I create a matrix for all features (```feature_matrix```), which shape is ```(STATE_SIZE, ACTION_SIZE, STATE_SIZE*ACTION_SIZE)``` and ```feature_matrix[i,j,:]``` is the feature vector for taking action ```j``` on state ```i```.

In [ ]:
s = torch.arange(STATE_SIZE).to(device)
s = s.unsqueeze(dim=-1).expand(-1,ACTION_SIZE)
a = torch.arange(ACTION_SIZE).to(device)
a = a.unsqueeze(dim=0).expand(STATE_SIZE,-1)
feature_matrix = get_features(s, a)
feature_matrix

tensor([[[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 0., 0

### 3. Create a transition probability matrix

As I have mentioned above, the algorithm (Maximum Causal Entropy IRL learner) should know the transition probabilities.

Now I generate a matrix with shape ```(STATE_SIZE, ACTION_SIZE, STATE_SIZE)```, in which the transition probability $p(s_k | s_i, a_j)$ is stored in the element of $(i,j,k)$, where $i, k = 0,1,\ldots,2499$ and $j=0,1,2,3$.

In this example, the environment has deterministic transition, where $p(s_k | s_i, a_j ) = 1.0$ for one successor state and $0.0$ for all others.

> Note : You can apply stochastic probability in this environment by setting ```transition_prob=True``` in constructor - in which, the action succeeds with probability `0.7`, a failure results in a uniform random transition (i.e, `0.1`, `0.1`, `0.1` respectively) to one of the adjacent states. (Change [this script](./00_generate_expert_trajectories.ipynb) and recreate expert's data.)<br>
> If you apply the stochastic transition probability, you should set the corresponding probabilities ($0 \lt p \lt 1$) in below matrics instead.

In [ ]:
states = torch.arange(STATE_SIZE).to(device)
states = states.unsqueeze(dim=-1).expand(-1,ACTION_SIZE)

actions = torch.arange(ACTION_SIZE).to(device)
actions = actions.unsqueeze(dim=0).expand(STATE_SIZE,-1)

next_states = env.step(actions.flatten(), states.flatten(), trans_state_only=True)
next_states = torch.reshape(next_states, (STATE_SIZE, ACTION_SIZE))
print(f"***** transition matrix (shape : ({STATE_SIZE}, {ACTION_SIZE}))*****")
print(next_states)

tran_probs = F.one_hot(next_states, num_classes=STATE_SIZE).float()
print(f"***** transition probability matrix (shape : ({STATE_SIZE}, {ACTION_SIZE}, {STATE_SIZE}))*****")
print(tran_probs)

***** transition matrix (shape : (2500, 4))*****
tensor([[   0,   50,    0,    1],
        [   1,   51,    0,    2],
        [   2,   52,    1,    3],
        ...,
        [2447, 2497, 2496, 2498],
        [2448, 2498, 2497, 2499],
        [2449, 2499, 2498, 2499]], device='cuda:0')
***** transition probability matrix (shape : (2500, 4, 2500))*****
tensor([[[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.]],

        [[0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.]],

        [[0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 

### 4. Create a function to get $V_{\theta,t}^{\verb|soft|}$ and $Q_{\theta,t}^{\verb|soft|}$

I create a function to build the following $V_{\theta, t}^{\verb|soft|}(s_t)$ and $Q_{\theta, t}^{\verb|soft|}(s_t, a_t)$ with current $\theta$.

- $\displaystyle V_{\theta, t}^{\verb|soft|}(s_t) = \log \sum_{a_t} \exp  Q_{\theta, t}^{\verb|soft|}(s_t, a_t) \;\;\; (\forall t = 0,1,\ldots,T-1) $
- $\displaystyle Q_{\theta, t}^{\verb|soft|}(s_t, a_t) = \theta^T \cdot \phi(s_t, a_t) + \gamma \mathbb{E}_{\mathcal{T}} \left[ V_{\theta,t+1}^{\verb|soft|}(S_{t+1}) | s_t, a_t \right] \;\;\; (\forall t = 0,1,\ldots,T-2) $
- $\displaystyle  Q_{\theta, T-1}^{\verb|soft|}(s_{T-1}, a_{T-1}) = \theta^T \cdot \phi(s_{T-1},a_{T-1}) $

The generated $V_{\theta, t}^{\verb|soft|}(S_t)$ is a matrix with shape ```(T, STATE_SIZE)```, and the generated $Q_{\theta, t}^{\verb|soft|}(S_t, A_t)$ is a matrix with shape ```(T, STATE_SIZE, ACTION_SIZE)```, where ```T``` is maximum time-step (i.e, horizon).

> Note : To prevent from overflow, use ```torch.logsumexp()``` for log-sum-exp (i.e, soft) computation.

In [ ]:
T = env.max_timestep
gamma = 1.0

def generate_soft_values(theta, tran_probs, feature_matrix, gamma):
    """
    Generate V and Q value's matrix. (See above.)

    Parameters
    ----------
    theta : torch.tensor((STATE_SIZE*ACTION_SIZE), dtype=float)
        Current reward's weight.
    tran_probs : torch.tensor((STATE_SIZE, ACTION_SIZE, STATE_SIZE), dtype=float)
        State transition probability matrix.
    feature_matrix : torch.tensor((STATE_SIZE, ACTION_SIZE, STATE_SIZE*ACTION_SIZE), dtype=float)
        All features for state and action pairs.
        For instance, the feature of state i and action j is feature_matrix[i, j, :],
        which shape is (STATE_SIZE*ACTION_SIZE).
    gamma : float
        Discount value (0 < gamma <= 1.0).

    Returns
    ----------
    V : torch.tensor((T, STATE_SIZE), dtype=float)
        Generated soft value matrix.
    Q : torch.tensor((T, STATE_SIZE, ACTION_SIZE), dtype=float)
        Generated soft Q-value matrix.
    """

    # initialize
    V = torch.zeros((T, STATE_SIZE)).to(device)
    Q = torch.zeros((T, STATE_SIZE, ACTION_SIZE)).to(device)
    # compute Q_{T-1}
    Q[T-1,:,:] = torch.sum(torch.mul(feature_matrix, theta), dim=-1)
    # loop for each T-1, T-2, ... , 0
    for t in reversed(range(T)):
        # compute Q_t
        if t != T-1:
            Q[t,:,:] = Q[T-1,:,:] + gamma * torch.sum(torch.mul(tran_probs,V[t+1,:]), dim=-1)
        # compute V_t
        V[t,:] = torch.logsumexp(Q[t,:,:], dim=-1)
    # return results
    return V, Q

### 5. Create a function to get policy $\pi$

With above $V_{\theta, t}^{\verb|soft|}(s_t)$ and $Q_{\theta, t}^{\verb|soft|}(s_t, a_t)$, you can get a time-dependant policy $\pi$ as follows.

$\displaystyle \pi_t(a_t | s_t) = \exp \left( Q_{\theta, t}^{\verb|soft|}(s_t, a_t) - V_{\theta, t}^{\verb|soft|}(s_t) \right)$

In this example, a function returns a matrix with shape ```(T, STATE_SIZE, ACTION_SIZE)```, in which the element of ```(i, j, k)``` is $\pi_i(k | j)$.

> Note : This always satisfies : $\sum_{a_t} \pi_t(a_t | s_t) = 1.0 \;\;\; \forall t, s_t$

In [ ]:
def generate_policy(v, q):
    return torch.exp(q - v.unsqueeze(dim=-1).expand(-1, -1, ACTION_SIZE))

### 6. Get expectation in expert policy

Now let's compute the following expectation by expert.<br>
This expectation is easily obtained by using expert trajectories.

$\displaystyle \mathbb{E}_{\pi^{\ast}} \left[ \sum_{t=0}^{T-1} \gamma^t \phi(S_t,A_t) \right]$

In [ ]:
import numpy as np

#
# Build states, actions, and mask in numpy array
#

states_all = []
actions_all = []
mask_all = []

# loop all trajectories in demonstration
current_timestep = 0
for timestep_len in timestep_lens:
    # pick up states and actions in a single trajectory
    states = exp_states[current_timestep:current_timestep+timestep_len]
    actions = exp_actions[current_timestep:current_timestep+timestep_len]

    # create mask (in which, TRUE in existing time-step, FALSE otherwise)
    mask = np.zeros(T, dtype=bool)
    mask[:len(states)] = True

    # fill (pad) states and actions
    states_traj = np.zeros(T, dtype=int)
    states_traj[:len(states)] = states
    actions_traj = np.zeros(T, dtype=int)
    actions_traj[:len(actions)] = actions

    # stack results
    states_all.append(states_traj)
    actions_all.append(actions_traj)
    mask_all.append(mask)

    # proceed to next trajectory
    current_timestep += timestep_len

# convert to numpy
states_all = np.stack(states_all, axis=0)
actions_all = np.stack(actions_all, axis=0)
mask_all = np.stack(mask_all, axis=0)

#
# Compute expectation in tensor
#

# convert to tensor
states_t = torch.from_numpy(states_all).to(device)
actions_t = torch.from_numpy(actions_all).to(device)
mask_t = torch.from_numpy(mask_all).to(device)

# Note : loop episode to prevent memory allocation error
episode_len = len(timestep_lens)
discount = torch.tensor([gamma**i for i in range(T)]).to(device)
sum_of_features = torch.zeros(STATE_SIZE*ACTION_SIZE).to(device)
for i in range(episode_len):
    # get feature vector
    features = get_features(states_t[i,:], actions_t[i,:])
    # get mask
    mask_i = mask_t[i,:]
    discount_i = discount * mask_i
    # compute the sum of discounted features
    sum_of_features += torch.sum(features * discount_i.unsqueeze(dim=-1).expand(-1,STATE_SIZE*ACTION_SIZE), dim=0)
# divide by the number of trajectories
feature_exp = sum_of_features / episode_len

### 7. Create a function to get expectation in learner policy $\pi$

Now I create a function to get the following expectation by $\pi$.

$\displaystyle \mathbb{E}_{\pi} \left[ \sum_{t=0}^{T-1} \gamma^t \phi(S_t,A_t) \right]$

In this example, I compute the expectation with state visitation frequecy in each time-step and by proceeding time-steps. (See below note.)

Probability of the initial state can be obtained by $\frac{1}{\verb| the number of possible initial states|}$.

> Note : An optimized algorithm (which computes state visitation frequencies) is often used to get feature expectation by policy. (See algorithm 9.3 in [original paper](https://www.cs.cmu.edu/~bziebart/publications/thesis-bziebart.pdf).)

In [ ]:
# compute initial state frequency
valid_states_2d = env.valid_states
valid_states_1d = valid_states_2d[:,0] * env.grid_size + valid_states_2d[:,1]
initial_state_prob = torch.zeros(STATE_SIZE).to(device)
initial_state_prob[valid_states_1d] = 1.0 / valid_states_1d.size(dim=0)

# create a function to get expectation in policy pi
def get_expectation_by_policy(pi, tran_probs, feature_matrix, gamma, initial_state_prob):
    current_state_prob = initial_state_prob
    flatten_feature_matrix = torch.reshape(feature_matrix, (STATE_SIZE*ACTION_SIZE, STATE_SIZE*ACTION_SIZE))
    # initialize expectation (the sum in all time-step)
    sum_of_features = torch.zeros(STATE_SIZE*ACTION_SIZE).to(device)
    # iterate time-step
    for t in range(T):
        # set zero as probability in goal state (because it's ended)
        current_state_prob[STATE_SIZE - 1] = 0.0
        # compute probability for taking action a on state s by :
        # [probability in current state s] * pi(a|s)
        s_a_prob = current_state_prob.unsqueeze(dim=-1).expand(-1,ACTION_SIZE) * pi[t,:,:]
        # compute the expectation at time-step t by summation of :
        # gamma^t * phi(s, a) * [above probability]
        average_feature = torch.sum(torch.flatten(s_a_prob).unsqueeze(dim=-1).expand(-1,STATE_SIZE*ACTION_SIZE) * flatten_feature_matrix, dim=0)
        sum_of_features += (gamma**t) * average_feature
        # update current state probability
        s_a_snext_prob = s_a_prob.unsqueeze(dim=-1).expand(-1,-1,STATE_SIZE) * tran_probs
        current_state_prob = torch.sum(torch.sum(s_a_snext_prob, dim=1), dim=0)
    return sum_of_features

### 8. Put it all together (Train parameters)

Finally, put it all together and then optimize parameter $\theta$.

In the middle of training, I want to check how $\pi_{\theta}$ is optimized.<br>
So, before optimizing parameters, I create a function for evaluation.

In [ ]:
def evaluate(env, pi, batch_size=128):
    total_reward = torch.tensor(0.0).to(device)
    s = env.reset(batch_size)
    for t in range(T):
        a_probs = pi[t,s,:]
        a = torch.multinomial(a_probs, num_samples=1).squeeze(dim=-1)
        s, r, term, trunc = env.step(a, s)
        total_reward += torch.sum(r)
        done = torch.logical_or(term, trunc)
        work_indices = (done==False).nonzero().squeeze(dim=-1)
        if not (len(work_indices) > 0):
            break;
        s = s[work_indices]
    return total_reward.item() / batch_size

Now let's run the optimization for $\theta$.

In [ ]:
learning_rate = 0.1

# initialize theta
theta = torch.empty(STATE_SIZE*ACTION_SIZE).to(device)
torch.nn.init.uniform_(theta, a=0.0, b=1.0)

prev_nabla_max = float("inf")
for iter_num in range(501):
    # get V and Q with current theta
    v, q = generate_soft_values(theta, tran_probs, feature_matrix, gamma)
    # get pi with current theta
    pi = generate_policy(v, q)
    # compute nabla
    feature_pi = get_expectation_by_policy(pi, tran_probs, feature_matrix, gamma, initial_state_prob)
    nabla = feature_pi - feature_exp
    # update theta (apply gradient descent)
    theta -= nabla * learning_rate
    # output logs
    nabla_mean = torch.mean(torch.abs(nabla)).item()
    nabla_max = torch.max(torch.abs(nabla)).item()
    print("iter {} - nabla average={:1.5f} max={:1.5f}, lr={:1.5f}".format(iter_num, nabla_mean, nabla_max, learning_rate), end="\r")
    if iter_num % 50 == 0:
        reward = evaluate(env, pi)
        print("\nestimated reward is {:.4f}.".format(reward))
    # adjust learning rate
    if prev_nabla_max < nabla_max:
        learning_rate /= 2.0
        prev_nabla_max = float("inf")
    else:
        prev_nabla_max = nabla_max

print("\nDone")

iter 0 - nabla average=0.02494 max=1.57862, lr=0.10000
estimated reward is -80.7109.

estimated reward is -9.8281.

estimated reward is -3.3359.

estimated reward is 0.7812.

estimated reward is 1.1953.

estimated reward is 2.2422.

estimated reward is 2.7422.

estimated reward is 3.8906.

estimated reward is 4.0391.

estimated reward is 4.6719.

estimated reward is 4.6328.

Done


## Show results

The following shows you which direction has high reward's weight in the trained parameters $\theta$.<br>
In this environment, the state in right-bottom corner is a goal state. (See [Readme.md](https://github.com/tsmatz/imitation-learning-tutorials/blob/master/Readme.md) for game rule in this environment.)<br>
As you can see below, the parameters are optimized to reach to the goal.

I note that the gray-colored cells are valid states, in which the agent can reach to the goal without losing any rewards.<br>
In this environment, the initial state is picked up from these gray-colored states.

In [ ]:
from IPython.display import HTML, display

# get all initial states
valid_states_all = torch.cat((env.valid_states, torch.tensor([env.grid_size-1,env.grid_size-1]).to(device).unsqueeze(dim=0)))
valid_states_all = valid_states_all[:,0] * env.grid_size + valid_states_all[:,1]

# create direction table
state_action_theta = torch.reshape(theta, (STATE_SIZE, ACTION_SIZE))
direction_table = torch.argmax(state_action_theta, dim=-1)
direction_table = torch.reshape(direction_table, (env.grid_size, env.grid_size))
direction_table = direction_table.cpu().numpy()

# show
html_text = "<table>"
for row in range(env.grid_size):
    html_text += "<tr>"
    for col in range(env.grid_size):
        if row*env.grid_size + col in valid_states_all:
            html_text += "<td bgcolor=\"gray\">"
            #
            # show direction
            #
            index = direction_table[row, col]
            if index == 0:
                html_text += "&#x2191;" # up
            elif index == 1:
                html_text += "&#x2193;" # down
            elif index == 2:
                html_text += "&#x2190;" # left
            elif index == 3:
                html_text += "&#x2192;" # right
        else:
            html_text += "<td>"
        html_text += "</td>"
    html_text += "</tr>"
html_text += "</table>"

display(HTML(html_text))

,,,,,,↓,↓,←,→,←,↓,,→,,,,,↓,,→,,←,→,↑,,→,→,→,,,,,←,→,↓,←,,,,,,,,,,,,,
,,,,,,↑,,↑,→,→,↓,↓,↓,→,↑,←,→,→,←,↓,,,,←,,←,↓,←,←,→,,→,,↑,,←,,,,,,,,,,,,,
,,,,,,→,,↓,,↓,→,→,←,←,→,→,,↑,→,,↑,↑,,←,↓,→,↑,←,→,↓,,→,←,,←,↑,,,↓,←,,,,,,,,,
,,,→,,,←,→,,↓,,→,→,←,←,,→,,→,,,,←,→,,,←,↑,,↓,↓,↓,←,←,↓,←,←,→,↑,↑,←,,,,,,,,,
,,,→,↑,→,←,→,↓,↑,→,←,↑,↑,→,←,↓,↓,→,,,↓,,↓,→,←,→,↓,→,→,→,→,→,←,↓,,↑,,←,↓,↓,,,,,,,,,
,,,←,←,,↑,,←,↑,,↑,←,,→,↓,,,→,↑,,↓,←,↓,,↓,←,→,↓,,,↑,←,,↓,,,,,←,↑,,,,,,,,,
,,↓,→,←,↑,↓,↓,→,←,→,↑,,,,↑,→,↓,←,←,,,↓,→,,,→,↑,,,,,→,↑,→,↓,→,↓,→,,,,,,,,,,,
↑,←,↓,→,,↓,↓,↑,,→,↑,←,↓,,←,→,,,→,↑,,↓,→,,←,↑,,←,←,←,,,↑,←,,↓,,↑,,,,,,,,,,,,
↓,↑,↑,→,→,↑,→,↓,↓,↓,,,→,,←,↑,,↓,↓,,,↑,↑,←,→,↑,→,,←,←,↓,,,,,,,→,,,,,,,,,,,,
→,→,←,,↑,,↑,↓,,,↓,↓,,→,↑,↓,←,←,↓,↓,→,→,,,,↓,,←,→,←,↑,←,→,→,→,←,↑,,,,,,,,,,,,,
↓,↓,←,←,←,,↓,→,→,↓,,→,↑,,←,↓,→,,↑,↓,↑,,→,↓,,→,→,←,,,↓,,,←,→,←,←,←,,,,,,,,,,,,←
